## Problem 1

The first part of this assignment consists in studying a single particle performing a continuous-time random walk in the network described by the graph in Fig. 1 (see text) and with the following transition rate matrix (see Lambda in the code below).

Your task is to simulate the particle moving around in the network in continuous time according to the transition rate matrix given.

In [ ]:
##### Setup
import numpy as np
from numpy.random import choice

# Define the mapping between 0,1,2,3,4 and o,a,b,c,d
mapping = {'o':0, 'a':1, 'b':2, 'c':3, 'd':4}

# Create Lambda
Lambda = [
    [0, 2/5, 1/5, 0, 0],
    [0, 0, 3/4, 1/4, 0],
    [1/2, 0, 0, 1/2, 0],
    [0, 0, 1/3, 0, 2/3],
    [0, 1/3, 0, 1/3, 0]]

# Define Q, the transition matrix associated to the jump chain associated to the CTMC
w = np.sum(Lambda, axis=1)
w_star = np.max(w)
Q = Lambda/w_star 
Q = Q + np.diag(np.ones(len(w))-np.sum(Q,axis=1))
print("Q matrix:\n", Q)

# Define the random walk simulation function
def random_walk(Q, xi, n_steps=100, until_first_return = False, until_S_hit=False, S=[0]):
    '''
    This function simulates a random walk of a particle starting in node xi of a graph with associated transition matrix Q
    
    Parameters:
        until_first_return: if True, the random walk stops when the particle has stepped out of xi at least once and then returns to xi
        until_S_hit: if True, the random walk stops when the particle hits a node contained in S
        n_steps, if both until_first_return and until_S_hit are false, the random walk stops when the particle has jumped n_steps times
    
    Returns:
        node_seq: sequence of nodes hit by the particle during its random walk
        transition_times: time instants in which the particle has jumped
    '''
    n_nodes = len(Q)
    node_seq = [xi]    # node_seq will keep trace of the visited states; start from node xi
    transition_times = [0]    # transition_times will store the time instants at which jumps/transitions happen; at t=0 the random agent is in node xi
    t_next = -np.log(np.random.rand())/w_star    # the random time to wait for the next transition is drawn from a rate-w_star exponential distribution
    x_start = xi    # starting node
    transition_time = 0    # t=0
    
    if until_first_return:
        while True:
            # Random jump:
            # the next state to visit will be extracted according to the probabilities
            # stored in the row of Q corresponding to the current state, node_seq[-1]
            old_xi = xi
            xi = choice(range(n_nodes), size=1, p=Q[xi])[0]
            if xi != old_xi:
                node_seq.append(xi)
                transition_time = transition_time + t_next    # store the time instant of the current transition
                transition_times.append(transition_time)
                t_next = -np.log(np.random.rand())/w_star
            else:    # do not jump, stay in the node: in this case, we still have to wait to jump
                t_next = t_next + -np.log(np.random.rand())/w_star
            if xi == x_start and transition_time!=0:    # stopping criterion
                return node_seq, transition_times
    
    elif until_S_hit:
        if x_start in S:
            return node_seq, transition_times
        while True:
            # Random jump:
            # the next state to visit will be extracted according to the probabilities
            # stored in the row of Q corresponding to the current state, node_seq[-1]
            old_xi = xi
            xi = choice(range(n_nodes), size=1, p=Q[xi])[0]
            if xi != old_xi:
                node_seq.append(xi)
                transition_time = transition_time + t_next    # store the time instant of the current transition
                transition_times.append(transition_time)
                if xi in S:    # stopping criterion: the particle has hit S
                    return node_seq, transition_times
                else:    # continue the simulation
                    t_next = -np.log(np.random.rand())/w_star
            else:    # in this case, we still have to wait to jump
                t_next = t_next + -np.log(np.random.rand())/w_star
                
    else:
        for _ in range(n_steps):
            # Random jump:
            # the next state to visit will be extracted according to the probabilities
            # stored in the row of Q corresponding to the current state, node_seq[-1]
            old_xi = xi
            xi = choice(range(n_nodes), size=1, p=Q[xi])[0]
            if xi != old_xi:
                node_seq.append(xi)
                transition_time = transition_time + t_next    # store the time instant of the current transition
                transition_times.append(transition_time)
                t_next = -np.log(np.random.rand())/w_star
            else:    # in this case, we still have to wait to jump
                t_next = t_next + -np.log(np.random.rand())/w_star
        return node_seq, transition_times


**a) What is, according to the simulations, the average time it takes a particle that starts in node a to leave the node and then return to it?**

In [ ]:
def estimate_avg_return_times(Q, n_rnd_walks=100):
    ''' This function estimates the average return times of the len(Q) = 5 nodes of the network '''
    n_nodes = len(Q)
    return_times = np.zeros(n_nodes)
    
    for node in range(n_nodes):    # for each node of G
        return_times_sample = []
        for _ in range(n_rnd_walks):
            _, transition_times = random_walk(Q, node, until_first_return = True)
            return_times_sample.append(transition_times[-1])
        return_times[node] = np.array(return_times_sample).mean()
    return return_times


# Compute the average return times to each of the five nodes
# (the simulation is performed 1000 times, so the average is computed over 1000 samples)
avg_return_times = estimate_avg_return_times(Q, n_rnd_walks=1000)
print("Return times:", avg_return_times)

# Get the average return time of node a
avg_return_time_a = avg_return_times[mapping['a']]
print("Return time of node a:", avg_return_time_a)


**b) How does the result in a) compare to the theoretical return time $\mathbb{E}_{a}[T^{+}_{a}]$?**

In [ ]:
# Find pi bar, the (unique) stationary probability vector
values,vectors = np.linalg.eig(Q.T)
index = np.argmax(values.real)
pi_bar = vectors[:,index].real
pi_bar = pi_bar/np.sum(pi_bar)
print("pi_bar =", pi_bar)

# Compute expected return times (using Kac's formula in continuous time)
expected_return_times = 1/(pi_bar * w)
print("Expected return times:", expected_return_times)

# Get the expected return time of node a
expected_return_time_a = expected_return_times[mapping['a']]
print("Return time of node a:", expected_return_time_a)

**c) What is, according to the simulations, the average time it takes to move from node o to node
d?**

In [ ]:
def estimate_avg_hitting_times(Q, S, n_rnd_walks=100):
    ''' This function estimates the average hitting times of the len(Q) = 5 nodes of the network related to the set of nodes S '''
    n_nodes = len(Q)
    hitting_times = np.zeros(n_nodes)

    for node in range(n_nodes):    # for each node of G
        hitting_times_sample = []
        for _ in range(n_rnd_walks):
            _, transition_times = random_walk(Q, node, until_S_hit=True, S=S)
            hitting_times_sample.append(transition_times[-1])
        hitting_times[node] = np.array(hitting_times_sample).mean()
    return hitting_times
    
S = [mapping['d']]
avg_hitting_times = estimate_avg_hitting_times(Q, S, n_rnd_walks=1000)
print("Average hitting times related to node d:", avg_hitting_times)

avg_hitting_time_o = avg_hitting_times[mapping['o']]
print("Average hitting time of node o related to node d", avg_hitting_time_o)

**d) How does the result in c) compare to the theoretical hitting-time $\mathbb{E}_{o}[T_{d}]$? (Describe also how
this is computed.)**

In [ ]:
# Define P, the transition matrix whose i-th row contains the transition probabilities from node i to its neighbors
# (no self-loops added, as in Q)
w = np.sum(Lambda, axis=1)    # node-clock rates
P = Lambda/(w.reshape(-1,1))    # normalization

# Compute expected hitting times related to d
A = np.eye(len(Q))-P    # (I-P) matrix of coefficients
A[4] = np.array([0,0,0,0,1])
b = 1/w    # (1/w_o, 1/w_a, ..., 1/w_d) vector of constant terms
b[4] = 0
expected_hitting_times = np.linalg.solve(A,b)
print("Expected hitting times related to node d:", expected_hitting_times)

# Get the expected hitting time of node o related to d
expected_hitting_time_o = expected_hitting_times[mapping['o']]
print("Expected hitting time of node o related to node d:", expected_hitting_time_o)